# A Crash Course in Parsing and Lexing




<!--
\index{parse tree}
\index{derivation}
-->
In this chapter we get our first glimpse of actual programming language implementation.
A good place to start with is the implementation of syntax analysis.
Every language processor in our language processor classification performs some sort of syntax analysis (with the exception of the generator).
As we saw in the previous chapter, the structure of a programming language can be captured by rule sets called grammars.
Therefore, we begin this chapter with a more detailed discussion of grammars including derivations and parse trees, and we
will take a look at how grammars are used to specify the syntax of programming languages.  We will use a grammar to 
specify our first programming language called Exp0.
 
 <!--
\index{parser}
\index{top-down parsing}
\index{bottom-up parsing}
\index{LL(k) parsing}
\index{LR(k) parsing}
-->
Grammars are extremely useful for specifying the structure of programming languages, but what makes them even more powerful as a
language specification tool is that we can easily convert grammars to programs that can recognize the structure of a programming
language, *i.e.*, we can convert grammars to programs that perform symbol grouping in an input stream according
to the syntax rules of the language.  Programs that recognize the structure of a programming language are called parsers.
The algorithms that parsers use for grouping symbols in an input stream come in two flavors: (a) top-down or LL(k) parsing and
(b) bottom-up or LR(k) parsing.  The discussion of grammars cannot be complete without talking about lexical analysis.
You can think of lexical analysis as a preprocessing step for parsing that break the symbol stream into discernable
tokens that can be processed by the grammar.

<!--
\index{parser generator}
\index{PLY}
-->
Programs that convert grammars into parsers are called parser generators.  Here and for the remainder of the book we will be using
a parser generator called PLY.  PLY is a bottom-up parser generator that reads a grammar specification and produces
a parser written in Python.  PLY actually contains both a parser generator and a lexical analyzer genration tool. We conclude the chapter with the construction of a couple of  language processors.  



In [1]:
import sys
sys.path.insert(0,"code")

# Grammars

## The Basics

<!--
\index{grammar}
\index{syntactic structure}
\index{grammar rule}
\index{context-free grammar}
\index{context-sensitive grammar}
\index{regular grammar}
-->
Grammars are sets of rules that define the syntactic structure of a programming language as we have seen in the rule
set in the last chapter.
Rules in grammars specify how symbols, words, and phrases are combined to make up valid sentences, *i.e.* programs.
When we talk about grammars in the context of specifying programming languages we usually mean context-free grammars as opposed to
context-sensitive or regular grammars, for example.
In context-free grammars the rules have a very specific form: there is only a single symbol on the left side of a rule and there are
zero or more symbols on the right side of a rule.  The fact that we allow a rule to have no symbols on the right means that a rule
allows us to derive *nothing* or in more meaningful syntactical terms, that a rule with no symbols on the right side 
allows us to derive the empty string. 
We often write the empty string explicitly rather than writing a rule with no symbols on the right side for easier readability. 
<!--
%For the remainder of the book we use the terms grammar and context-free grammar interchangeably with the understanding that we 
%always mean context-free grammars.
-->

<!--
\index{arithmetic expression}
-->
Consider the following  grammar that specifies the syntactic structure of arithmetic expressions with variables  `x`,
`y`, and `z`:
```
Sentence : Expression
Expression : Expression + Expression
           | Expression - Expression
           | Expression * Expression
           | Expression / Expression
           | ( Expression )
           | x
           | y
           | z
```
The grammar states that a sentence in this language is an expression and expressions can be add, subtract, multiply, or divide expressions
as well as parenthesized expressions and variable names.
Note that the parentheses are part of the language that we are defining, that is, the parentheses are part of the syntax of the language.
Also note that the '`|`' operator denotes alternatives in production as in: an expression can be a addition expression
or a subtraction expression *etc*.

<!--
\index{derivation}
\index{valid sentence}
-->
A grammar allows us to derive any valid sentence in the language that it defines by applying the rules to symbols appearing
in the derivation until no further application is possible.
In our case,
we can derive the sentence `x + y` from the symbol `Sentence` in the above grammar as follows,
```
Sentence =>
Expression =>
Expression + Expression =>
x + Expression =>
x + y
```
That means, the sentence `x + y` is a valid sentence in this language.

<!--
\index{grammar production}
\index{non-terminal (symbol)}
\index{terminal (symbol)}
\index{start symbol}
-->
We associate specific terminology with grammars.
The rules are called *productions* and the symbols appearing on the left sides of productions are called 
*non-terminals.*
The non-terminal set for the grammar above is,
 ```
 { Sentence, Expression }.
 ```
Any symbol appearing in the grammar productions that is not part of the non-terminal set,
*i.e.*, any symbol that does not appear on the
left side of a production, is called a *terminal*.  The terminal set of our grammar is,
```
{ +, -, *, /, (, ), x, y, z }.
```
The terminals make up the syntax of a programming language.
Note that the parentheses are part of the terminal set, that is, they are part of the language that we are defining.

Each grammar has one non-terminal that is always used to start all the derivations in that grammar. That non-terminal is called the *start symbol*.
In our grammar the symbol `Sentence` is the start symbol.  It is a convention that the start symbol is the non-terminal defined by the
first rule in the grammar, that is, the start symbol is the symbol on the left side of the first rule of the grammar.  We can summarize grammar terminology as follows,

>A grammar consists of the following:
* A set of productions which are rules with a single symbol on the left and zero or more symbols on the right.
* A symbol on the left side of a production is called a non-terminal.
* Any symbol in the grammar that is not a non-terminal is called a terminal.
* We have a special non-terminal symbol called the start-symbol.




## Derivations

<!--
\index{derivation}
\index{valid sentence}
-->
Given our grammar terminology, we can now be precise of what we mean by a derivation and valid sentence,

>A derivation is a sequence of steps that begins with the start symbol and at 
each derivation step replaces a single non-terminal 
with the right side of a production that has that non-terminal on the left side.
A valid sentence in the language of a grammar is a sequence of symbols that contains only terminals and is arrived at through a derivation
.

<!--
\index{left-most derivation}
\index{right-most derivation}
\index{parsing algorithm}
-->
Let us use the grammar defined above that specifies simple arithmetic expressions and show that the sentence 
```
x * (y + z)
```
is a valid sentence in the language of that grammar.
In order to show that it is a valid sentence we have to show that we can derive it from the start symbol `Sentence`:
```
Sentence =>
Expression =>
Expression * Expression =>
x * Expression =>
x * ( Expression ) =>
x * ( Expression + Expression ) =>
x * ( y + Expression ) =>
x * ( y + z )
```
Here the last derivation step shows that `x * (y + z)` is a valid sentence.
If you look closely at the derivation then you will discover that we 
have consistently expanded the left-most non-terminal at each step of the derivation.  
This kind of derivation is called the *left-most derivation*.
Consistently expanding the right-most non-terminal at each step of a derivation is called the *right-most derivation*,
```
Sentence =>
Expression =>
Expression * Expression =>
Expression * ( Expression ) =>
Expression * ( Expression + Expression ) =>
Expression * ( Expression + z ) =>
Expression * ( y + z ) =>
x * ( y + z )
```
If a sentence is valid then it does not matter which derivation we chose to derive it from the start symbol, both derivation techniques will
derive the same sentence as long as we make the same choices in terms of which rules to apply.
Later on we will see that we classify parsing algorithms according to the derivation they construct for a program.




## Parse Trees

---
<center>
<img src="figures/chap02/1/figure/Slide1.jpg" alt="">
Fig 1. A parse tree for the expression `x + y * z`.
</center>

---

<!--
\index{parse tree}
\index{derivation}
-->

Derivations can be represented as tree structures. 
Consider the left-most derivation of the sentence,
```
x + y * z
```
using our grammar for arithmetic expressions from above we can construct the following derivation for
this sentence,
```
Sentence =>
Expression =>
Expression + Expression =>
x + Expression =>
x + Expression * Expression =>
x + y * Expression =>
x + y * z 
```
We construct a parse tree for this derivation by making the start symbol `Sentence` the root node of the tree.
Then for every non-terminal that we replace
in the derivation we write the right side of the rule that we use to replace the non-terminal underneath it in
the tree and connect all the symbols that we just
placed underneath the non-terminal to the non-terminal symbol itself.
Figure 1 shows the parse tree constructed in this way for the derivation above.
The leaves of the parse tree spell out the sentence that we wanted to derive and we highlighted this by placing the derived sentence into
a box at the bottom of the figure.
We can now say the following,

> A parse tree is a tree based representation of a derivation in a grammar.

The following movie is an animation of the parse tree construction for the expression `x + y * z`.

<!-- parse tree construction video -->
<a href="http://www.youtube.com/watch?feature=player_embedded&v=vo8HBQDi6bk" target="_blank">
<img style='border:1px solid #000000' src="movie.jpg" width="120" height="90" />
</a>

<!--
\index{parser}
\index{parse tree}
-->
An interesting and very desirable characteristic of parse trees is that they make the grouping of the symbols in an input stream
structurally explicit.
In Figure 1  we can see that `y * z` are grouped together as an expression and this expression together
with the symbols `x` and `+` forming another expression.
This hierarchical grouping of symbols, words, and phrases in parse trees makes language processing much easier compared to processing textual representations of programs.

Another observation is that the same parse tree is obtained regardless whether we use a left-most or right-most derivation of
our sentence.  Try to prove to yourself that a right-most derivation of our sentence gives rise to the same parse tree
(the first three steps in the right-most derivation are obviously the same as in the left-most derivation above).

## An Example: The Exp0 Language

<!--
\index{Exp0}
\index{Lisp}
-->

Our first programming language  is a calculator language that allows you to evaluate, store, and print the values of simple expressions -
let us call it Exp0.
In order to keep it simple we only allow three variable names (`x`, `y`, and `z`) and our
numbers are limited to single digits.
The expressions in this language are in prefix format, *e.g.*, the expression `(+ x 1)` means add the value `1` to the
the value stored in variable `x`.
If you know Lisp then this should look very familiar; in Lisp all
expressions as well as statements are given in this prefix format.
For the sake of simplicity we also only allow additive expressions, that is, we only allow addition and subtraction.
To print a value to the terminal we write the command `p` followed by an expression.
For example, the statement `p (+ x 1)`
adds the value one to the value stored in `x`  and then prints that new value to the terminal.
We use the command `s` followed by a variable name followed by an expression to store the value of
the expression in the variable.
Here, the statement `s y (- 9 3)` stores the value six in the variable `y`.

<!--
\index{Exp0 grammar}
-->

Here is the grammar that defines the syntax of Exp0,
```
prog : stmt_list

stmt_list : stmt stmt_list
          | ""
          
stmt : p exp ;
     | s var exp ;
     
exp : + exp exp
    | - exp exp
    | ( exp )
    | var
    | num
	
var : x | y | z

num : 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |9
```
The non-terminal `prog` is the start symbol.  The first rule defines the start symbol and states that programs are statement lists.
The second and the third rules define statement lists.
The rules for the non-terminal `stmt` define the structure of print and store commands.
Note the semicolon that terminates these statements.  These semicolons belong to the Exp0 language we are defining.
The rules for the non-terminal `exp` specify the structure of possibly parenthesized prefix expressions.
As is common, variable names and numbers are also considered expressions.
Finally, the last two sets of rules
define our variable names and our numbers.




---
<center>
<img src="figures/chap02/2/figure/Slide1.jpg" alt="">
Fig 2. A parse tree for the Exp0 program `s x 1; p (+ x 1);`.
</center>

---




Let us use our Exp0 grammar to derive the program
```
s x 1 ;  p (+ x 1) ;
```
In this program we first store the value one in variable `x` and then we 
print out the sum of the value stored in `x` and the value one.
Our left-most derivation starts with the start symbol `prog`,
```
prog =>
stmt_list =>
stmt  stmt_list =>
s var exp ; stmt_list =>
s x exp ; stmt_list =>
s x num ; stmt_list =>
s x 1 ; stmt_list =>
s x 1 ; stmt stmt_list=>
s x 1 ; p exp ; stmt_list =>
s x 1 ; p ( exp ) ; stmt_list =>
s x 1 ; p ( + exp exp ) ; stmt_list =>
s x 1 ; p ( + var exp ) ; stmt_list =>
s x 1 ; p ( + x exp ) ; stmt_list =>
s x 1 ; p ( + x num ) ; stmt_list =>
s x 1 ; p ( + x 1 ) ; stmt_list =>
s x 1 ; p ( + x 1 ) ; 
```
The recursive nature of rules 2 and 3 of our Exp0 grammar allows us to derive sequences of statements
that make up a program.
In the last derivation step we use the rule `stmt_list : ""` to let the non-terminal "disappear".
The parse tree for this derivation appears in Figure 2.
You should convince yourself that this is indeed the parse tree for the above derivation.




# Parsers

<!--
\index{parser}
\index{derivation}
\index{valid sentence}
-->


In the previous sections we used our intuition to select grammar rules that make up a derivationa that prove that some sentence is valid in the language of a grammar.
Here we investigate the algorithmic construction of derivations for sentences in a particular grammar.
Programs that perform the construction of derivations are called parsers.
We begin our discussion with the top-down parsing algorithm.


## Top-Down Parsing

It is highly likely that when you construct the derivation of a particular sentence you first 
scan the structure of the sentence itself and then you scan
the right sides of the grammar rules. 
Once you have an overview of the structure of the right sides of the grammar rules you will most likely pick  rules that match some
structure in the sentence that you want to derive.  
You keep doing this until you derive the terminals that make up the sentence.

<!--
\index{lookahead set}
-->

Top-down parsing mimics this approach.
However, in order to make this workable algorithmically we need to extend grammars with an additional data structure called *lookahead sets*.
Our Exp0 grammar extended with lookahead sets looks like this,
```
prog : {p,s} stmt_list

stmt_list : {p,s} stmt stmt_list
          | {""} ""
          
stmt : {p} p exp ;
     | {s} s var exp ;
     
exp : {+} + exp exp
    | {-} - exp exp
    | {(} ( exp )
    | {x,y,z} var
    | {0,1,2,3,4,5,6,7,8,9} num
	
var : {x} x | {y} y | {z} z

num : {0} 0 | {1} 1 | {2} 2 | {3} 3 | {4} 4 | {5} 5 | {6} 6 | {7} 7 | {8} 8 | {9} 9
```
Here the lookahead sets appear between the curly braces for each rule.  

<!--
\index{lookahead pointer}
-->

From the point of view of top-down parsing the lookahead sets summarize the structure
of the right side of each grammar rules and they are used  in conjunction with 
a lookahead pointer to make choices between competing rules during the construction of a derivation.
The  lookahead pointer points to the current character in the input stream.
Consider the following input stream:
```
<p> + 1 2 ; \eof
```
The angle brackets around the `p` character represent the lookahead pointer.
Now consider the left-most derivation of this program using our grammar extended with the lookahead sets.
In the derivation below we show the input stream, the derivation in the square brackets,
and the rule which we apply.
We start with the character under the lookahead pointer in the stream and with our start symbol `prog`.
We can observe that the lookahead symbol matches the lookahead set for the rule defining `prog`
```
<p> + 1 2 ; \eof    [<prog>]                     prog : {p,s} stmt_list =>
<p> + 1 2 ; \eof    [<stmt_list>]                stmt_list : {p,s} stmt stmt_list =>
<p> + 1 2 ; \eof    [<stmt> stmt_list]           stmt : {p} p exp ; =>
<p> + 1 2 ; \eof    [<p> exp ; stmt_list]        <match input character> =>
p <+> 1 2 ; \eof    [p <exp> ; stmt_list]        exp : {+} + exp exp =>
p <+> 1 2 ; \eof    [p <+> exp exp ; stmt_list]  <match input character> =>
p + <1> 2 ; \eof    [p + <exp> exp ; stmt_list]  exp : {0,1,2,3,4,5,6,7,8,9} num =>
p + <1> 2 ; \eof    [p + <num> exp ; stmt_list]  num : {1} 1 =>
p + <1> 2 ; \eof    [p + <1> exp ; stmt_list]    <match input character> =>
p + 1 <2> ; \eof    [p + 1 <exp> ; stmt_list]    exp : {0,1,2,3,4,5,6,7,8,9} num =>
p + 1 <2> ; \eof    [p + 1 <num> ; stmt_list]    num : {2} 2 =>
p + 1 <2> ; \eof    [p + 1 <2> ; stmt_list]      <match input character> =>
p + 1 2 <;> \eof    [p + 1 2 <;> stmt_list]      <match input character> =>
p + 1 2 ; <\eof>    [p + 1 2 ; <stmt_list>]      stmt_list : "" =>
p + 1 2 ; <\eof>    [p + 1 2 ;] 
```
At the final step of the derivation the lookahead pointer points to the `\eof` symbol, there are no non-terminals left in the derivation, and the derived sentence matches
the program in the input stream: We say that we successfully parsed the input stream.

<!-- TODO: more detailed explanation of the derivation -->


---
<center>
<img src="figures/chap02/3/figure/Slide1.jpg" alt="">
Fig 3. A parse tree for the Exp0 program `p + 1 2;`.
</center>

---



<!--
\index{top-down parsing}
-->

The parse tree for our derivation is shown in Figure 3.
If you look carefully at the derivation then you can see the it constructs the elements of the parse tree starting
with the root not.
Therefore his is called top-down parsing.

<!--
\index{LL(1)}
\index{LL(1)!parsing}
\index{LL(1)!grammar}
-->

Another name for this approach to parsing is *LL(1)* parsing.  The first L stands for the fact that we are reading the input stream
from left to right.  The second L indicates that we are performing a left-most derivation, and (1) means that we are using a
single lookahead symbol in the input stream

>Top-down parsing means building a derivation or a parse tree starting with the start symbol.  Another name for this is LL(1) parsing which means
reading from the (L)eft constructing the (L)eft-most derivation using (1) lookahead symbol. 
Grammars that allow us to do LL(1) parsing are called LL(1) grammars.

The key to LL(1) parsing are the lookahead sets.



### Lookahead Sets

<!--
\index{lookahead set}
-->

You probably noticed that the introduction of the lookahead pointer and the lookahead sets made constructing derivations very mechanical.
There is no guess work in terms of which rule to apply when constructing a derivation.
This is just what we need in order to have machines do this.

The first step in mechanizing this whole process is the construction of the lookahead set.
Here is the Python code that given a grammar encoded as an appropriate data structure will
generate the lookahead set for each of the rules of the grammar.



In [2]:
from grammar_stuff import first_symbol, terminal_set, non_terminal_set
import pprint
pp = pprint.PrettyPrinter()

In [3]:
def compute_lookahead_sets(G):
    '''
    Accepts: G is a context-free grammar viewed as a list of rules
    Returns: GL is a context-free grammar extended with lookahead sets
    '''
    GL = []
    for R in G:
        (A, rule_body) = R
        S = first_symbol(rule_body) 
        if S == "":
            GL.append((A, set([""]), rule_body)) 
        elif S in terminal_set(G):
            GL.append((A, set(S), rule_body)) 
        elif S in non_terminal_set(G):
            L = lookahead_set(S,G)
            GL.append((A, L, rule_body))
    return GL

This function iterates over the rules of a grammar and looks at the first symbol of each rule body.
If the first symbol is either the empty string or a terminal symbol the function just adds the first symbol
as the lookahead set for that rule.  If the first symbol of the rule is a non-terminal then the function computes
the lookahead set for that non-terminal with the function `lookahead_set`.

In [4]:
def lookahead_set(N, G):
    '''
    Accepts: N is a non-terminal in G
    Accepts: G is a context-free grammar
    Returns: L is a lookahead set
    '''
    L = set()
    for R in G:
        (A, rule_body) = R
        if A == N:
            Q = first_symbol(rule_body)
            if Q == "":
                raise ValueError("non-terminal {} is a nullable prefix".format(A))
            elif Q in terminal_set(G):
                L = L | set(Q)
            elif Q in non_terminal_set(G):
                L = L | lookahead_set(Q, G)
    return L

The function `lookahead_set` iterates over rules of the grammar `G` that have the non-terminal `N` on the
left side.  If the first symbol of the rule body of one of these rules is the empty string then we abort.
Our simple algorithm cannot deal with nested rules that all derive the empty string.
On the other hand, if the first symbol is a terminal then we add this terminal to the lookahead set.  If the
first symbol is a non-terminal then we call ourselves recursively in order to determine the lookahead set for that
non-terminal.

Let's test these functions using the following grammar G:
```
exp : + exp exp
    | - exp exp
    | x
    | y
    | z
```
We encode the grammar in Python as a list of tuples where each tuple represents a grammar rule in form:
(left side, right side). The right side is a list of symbols occuring in the right side of that grammar rule.

In [5]:
G = [('exp',['+','exp','exp']), 
     ('exp',['-','exp','exp']), 
     ('exp',['x']), 
     ('exp',['y']), 
     ('exp',['z'])]

Let's compute the terminals and non-terminals of our grammar.

In [6]:
print(terminal_set(G))

{'y', 'x', 'z', '+', '-'}


In [7]:
print(non_terminal_set(G))

{'exp'}


Now, let's compute the extended grammar with the lookahead sets.

In [8]:
pp.pprint(compute_lookahead_sets(G))

[('exp', {'+'}, ['+', 'exp', 'exp']),
 ('exp', {'-'}, ['-', 'exp', 'exp']),
 ('exp', {'x'}, ['x']),
 ('exp', {'y'}, ['y']),
 ('exp', {'z'}, ['z'])]


### Left-Recursive Grammars are not LL(1)

<!--
\index{left-recursive grammar}
\index{mutual left-recursion} 
-->


There are grammars for which it is impossible to construct lookahead sets.
Consider the following grammar,
```
exp : exp + exp
    | x
    | y
```
Our lookahead set algorithm will recurse indefinitely on the first rule of this grammar (convince yourself!).
The indefinite recursion is due to the fact that the first non-terminal on the right side of the rule is exactly the 
same non-terminal on the left side of the rule.  This is called *left-recursion* and grammars that exhibit left-recursion in their rules cannot be used for LL(1) parsing. We say,

> Left-recursive grammars are not LL(1).

You have to be careful, left-recursion is not always immediately apparent.  Consider the following grammar with
mutually recursive rules,
```
exp : term + var
    | var
    
term : exp + var
     | var
     
var : x
    | y
```
None of the rules in this grammar is immediately left-recursive.  However, the mutual left-recursion of the first and third rule
is problematic and again our lookahead set algorithm will recurse indefinitely showing that this grammar is not LL(1).
Later on we will see techniques that allow us to rewrite left-recursive grammars into grammars that are LL(1).



### Other Grammars that are not LL(1)

<!--
\index{rule!prefix}
\index{rule!factoring}
-->

There is another class of grammars that is not considered to be LL(1); grammars where the prefixes of the right
sides of rules overlap.
Consider this grammar,
```
exp : + exp exp
    | term
    
term : var [ exp ]
     | var
     
var : x | y
```
Notice that in this grammar the two rules defining the non-terminal `term` both start with the non-terminal `var`; we say that the two rules share a *prefix*.
This is a problem because both rules will have the same lookahead set which prevents LL(1) parsing from being able to
choose a rule unambiguously. We say,

>Grammars that have rules defining the same non-terminal and that share a common prefix are not LL(1).

Turns out there is an easy fix.  We can rewrite the grammar above as follows,
```
exp : + exp exp
    | term
    
term : var array

array : [ exp ]
      | ""
      
var : x | y
```
This technique is called *rule factoring* and is often used to turn non-LL(1) grammars into LL(1) grammars.

<!--
\index{nullable prefix}
-->

There is one additional class of grammars that are not considered LL(1) according to our lookahead set computation.
These are grammars that have rules with right sides that start with non-terminals deriving the empty string.
Consider the following grammar,
```
A : B a
  | c
  
B : b
  | ""
```
Without the test  in our algorithm rejecting these kinds of grammars our lookahead set would look something like this for the grammar above,
```
A : {b,""} B a
  | {c} c
  
B : {b} b
  | {""} ""
```
Here the empty string as a lookahead symbol for the first rule does not make any sense because the right side of that rule
is not the empty string and therefore we reject this grammar.
We could of course extend out lookahead set algorithm to accommodate this empty string and compute the
lookahead set for the first rule as `{ a, b}` but the algorithm that does this in the general case
is too complicated for our purposes considering that there is an easy rewrite of the grammar,
```
A : b a
  | a
  | c
```
Grammars with rules whose right sides start with non-terminals that derive the empty string are said to contain *nullable prefixes*.  Therefore, according to our definition of lookahead sets and for our purposes we say that,

> In our setting grammars with nullable prefixes are not LL(1).

<!--
\index{LL(1)!language}
-->

The fact that we can have multiple grammars that define the same language is interesting in itself.  The fact that some of these grammars are LL(1)
and some are not is even more interesting and leads to the following observation,

> A language is called LL(1) if it has at least one LL(1) grammar.





### A Top-Down Parsing Algorithm

<!--
\index{top-down parsing}
-->

Now that we know how to construct grammars that have  lookahead sets we can examine a simple top-down parsing algorithm itself.
The algorithm takes an input stream and a context-free grammar extended with lookahead sets as inputs and returns `True` or `False`
depending on whether the program in the input stream is a valid sentence in language of the grammar or not.

In [9]:
from grammar_stuff import terminal_set, non_terminal_set, start_symbol, find_matching_rule
from grammar_stuff import Stack, InputStream

In [10]:
def topdown_parse(I, GL):
    # Accepts: I is an input stream 
    # Accepts: GL is an LL(1) grammar extended with lookahead sets  
    # Returns: True or False
    S = Stack()
    S.push(start_symbol(GL))
    while not S.empty():
        P = I.pointer()
        A = S.peek()
        if A in non_terminal_set(GL):
            rule = find_matching_rule(GL, A, P)
            if not rule:
                return False
            else:
                S.pop()
                (A, L, rule_body) = rule
                S.push_reverse(rule_body)
        elif A in terminal_set(GL):
            if A == P:
                S.pop()
                I.next()
            elif A == "":
                S.pop()
            else:
                return False

    if I.end_of_file():
        return True
    else:
        return False

Internally the algorithm uses a stack to keep track of the derivations and essentially mechanizes the derivation we
constructed in section 'Top-Down Parsing'.

Let's test our parsing algorithm on our simple grammar from before,
```
exp : + exp exp
    | - exp exp
    | x
    | y
    | z
```
As before, we encode our grammar as a list of tuples where each tuple represents a rule,

In [11]:
G = [('exp',['+','exp','exp']), 
     ('exp',['-','exp','exp']), 
     ('exp',['x']), 
     ('exp',['y']), 
     ('exp',['z'])]

Next we set up our input stream and compute the grammar `GL` which is the grammar `G` enriched with the
lookahead sets,

In [12]:
I = InputStream(['+', 'x', 'y'])

GL = compute_lookahead_sets(G)

We are now ready to parse our input stream `I` using the extended grammar `GL`,

In [13]:
print(topdown_parse(I, GL))

True


Success! It is clear that the input stream represents a valid sentence and parsing it confirms it.

Let's try this with our Exp0 language.  Here is the grammar of Exp0 rewritten slightly so that our
lookahead set algorithm can process it,
```
prog : stmt prog
     | ""
          
stmt : p exp ;
     | s var exp ;
     
exp : + exp exp
    | - exp exp
    | ( exp )
    | var
    | num
	
var : x | y | z

num : 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |9
```
Encoding this grammar as a tuple list,

In [14]:
G = [
    ('prog',['stmt','prog']),
    ('prog',[""]),
    ('stmt',['p','exp',';']),
    ('stmt',['s','var','exp',';']),
    ('exp',['+','exp','exp']),
    ('exp',['-','exp','exp']),
    ('exp',['(','exp',')']),
    ('exp',['var']),
    ('exp',['num']),
    ('var',['x']),
    ('var',['y']),
    ('var',['z']),
    ('num',['0']),
    ('num',['1']),
    ('num',['2']),
    ('num',['3']),
    ('num',['4']),
    ('num',['5']),
    ('num',['6']),
    ('num',['7']),
    ('num',['8']),
    ('num',['9'])
]

Computing the lookahead sets,

In [15]:
GL = compute_lookahead_sets(G)
pp.pprint(GL)

[('prog', {'s', 'p'}, ['stmt', 'prog']),
 ('prog', {''}, ['']),
 ('stmt', {'p'}, ['p', 'exp', ';']),
 ('stmt', {'s'}, ['s', 'var', 'exp', ';']),
 ('exp', {'+'}, ['+', 'exp', 'exp']),
 ('exp', {'-'}, ['-', 'exp', 'exp']),
 ('exp', {'('}, ['(', 'exp', ')']),
 ('exp', {'y', 'z', 'x'}, ['var']),
 ('exp', {'2', '4', '0', '3', '5', '6', '7', '1', '9', '8'}, ['num']),
 ('var', {'x'}, ['x']),
 ('var', {'y'}, ['y']),
 ('var', {'z'}, ['z']),
 ('num', {'0'}, ['0']),
 ('num', {'1'}, ['1']),
 ('num', {'2'}, ['2']),
 ('num', {'3'}, ['3']),
 ('num', {'4'}, ['4']),
 ('num', {'5'}, ['5']),
 ('num', {'6'}, ['6']),
 ('num', {'7'}, ['7']),
 ('num', {'8'}, ['8']),
 ('num', {'9'}, ['9'])]


Finally, let's set up the input stream for the program `p + 1 2 ;`

In [16]:
I = InputStream(['p','+','1','2',';'])

Is it a valid sentence in the language of the grammar `G`?

In [17]:
print(topdown_parse(I, GL))

True


Figure 4 shows a trace of our top-down parsing algorithms at work on the example that we just parsed.
The figure shows the stack, the input stream with the stream pointer shown in angle brackets, and the rules
as they are being applied.

---
<center>
<img src="figures/chap02/4/figure/Slide1.jpg" alt="">
Fig 4. A trace of our top-down parsing algorithm  using the extended grammar `GL` for the Exp0 language
 the input stream `p + 1 2 ; \eof`.
</center>

---




Compare this derivation to the derivation that we constructed by hand for this input stream in section 'Top-Down Parsing'.  You will find them virtually indentical with the exception that we had to make some changes to the grammar to accommodate our lookahead set computation.  That means our top-down parsing algorithm mechanizes our derivations
very nicely.

<!-- TODO patch up the movie for this - videos/chap02/q2 -->

The following is an animation of the derivation process using our algorithm.

<a href="http://www.youtube.com/watch?feature=player_embedded&v=DAnlgrbSpyQ" target="_blank">
<img style='border:1px solid #000000' src="movie.jpg" width="120" height="90" />
</a>



## Bottom-Up Parsing

<!--
\index{bottom-up parser}
-->

In the previous sections we have looked at top-down parsers which conceptually build parse trees top-down starting at the root.
As you might guess, bottom-up parsers do exactly the opposite, rather than building a parse tree starting from the root, a bottom-up parser
constructs a parse tree starting with the leaves of the parse tree working its way up to the root node. 
A bottom-up parser also accepts an input stream `I` and a grammar `G` as arguments and uses
a stack `S` internally for processing.
However, it is worthwhile pointing out that in this case the grammar does not have to be a LL(1) grammar and we do not have to compute
the lookahead sets.



<!--
\index{reduce action}
\index{shift action}
\index{accept action}
\index{reject action}
-->

Traditionally bottom-up parsers are understood in terms of four different actions,

1. **Reduce**: Apply a grammar rule to the stack.
2. **Shift**: Push a new symbol from the input stream onto the stack
3. **Accept**: Indicate that the string in the input stream is a valid sentence in language of the grammar.
4. **Reject**: Indicate that the string in the input stream is not a valid sentence in the language of the grammar.

The following algorithm shows these actions as comments for the different pieces of code
accomplishing the respective tasks.
If you look at the code carefully then you will notice that the algorithm uses the rules of the grammar backwards ('Reduce') compared to what we
saw in the top-down parser.
Here the parse will try to match the right side of the rule with the contents on the top of the stack.  If successful
then the stack is popped and the corresponding non-terminal is pushed.

### A Bottom-Up Parsing Algorithm

Here is a stack based bottom up parsing algorithm:

In [18]:
from grammar_stuff import right_side_match, start_symbol
from grammar_stuff import Stack, InputStream

In [19]:
def bottomup_parse(I, G):
    # Accepts: I is an input stream
    # Accepts: G is a context-free grammar
    # Returns: True or False
    S = Stack()
    while True:
        rule = right_side_match(G, S)
        if rule: # Reduce
            (Q, rule_body) = rule
            S.popn(len(rule_body))
            S.push(Q)
        elif not I.end_of_file(): # Shift
            S.push(I.pointer())
            I.next()
        elif S.peek() == start_symbol(G) and I.end_of_file(): # Accept
            return True
        else: # Reject
            return False

Let's test our parser on a grammar that our top-down approach could not handle: the left-recursive grammar
that specifies simple expressions,
```
exp : exp + exp
    | x
    | y
```
We encode this grammar as our familiar list of tuples,

In [20]:
G = [
    ('exp',['exp','+','exp']),
    ('exp',['x']),
    ('exp',['y'])
]

We also need to set up our input stream with the program `x + y`,

In [21]:
I = InputStream(['x','+','y'])

Let's parse this stream,

In [22]:
print(bottomup_parse(I, G))

True


Success! As expected the program `x + y` does belong to the language of the grammar.

Let's look at a trace of this parsing algorithm parsing the input stream `x + y \eof`.  Figure 5 shows this trace.

---
<center>
<img src="figures/chap02/5/figure/Slide1.jpg" alt="">
Fig 5. A trace of our bottom-up parsing algorithm  for the input stream `x + y \eof`.
</center>

---


<!--
\index{right-most derivation}
\index{LR(0) parser}
-->

A scan of the Rule column of Figure5  reveals a couple of interesting things:

1. The algorithm seems to be using the rules of the grammar in exactly the opposite order compared to the top-down parsing algorithm.

2. It is not difficult to see that this algorithm constructs the parse tree from the bottom up: it first constructs two subtrees using the rules `exp : x` and `exp : y` and then combines these two subtrees into a single tree using the rule `exp : exp + exp`.

3. If we scan the Stack column from bottom to top we can see that the algorithm constructs a right-most derivation,
```
exp =>
exp + exp =>
exp + y =>
x + y
```

This parsing algorithm did not use the lookahead pointer in order to make decisions
as to which rules to apply to the stack.  It simply used the lookahead pointer as a way to remember where it is in the input stream.
Because it does not use the lookahead pointer to make parsing decisions we say that the algorithm uses (0) lookahead symbols.
Also note that the algorithm reads the input stream from the left but constructs the right-most derivation.
We can now say the following,

> Bottom-up parsers build parse trees starting from the leaves and work their way towards the root node.
This is also called LR parsing.
In our case we have LR(0) parsing because we are reading from the (L)eft constructing the (R)ight-most
derivation using (0) lookahead symbols.
Grammars that allow us to do LR(0) parsing are called LR(0) grammars.

Here is an animation of our bottom-up parsing algorithm.
<!-- videos/chap02/q3 -->

<a href="http://www.youtube.com/watch?feature=player_embedded&v=tC7allwJZlM" target="_blank">
<img style='border:1px solid #000000' src="movie.jpg" width="120" height="90" />
</a>



### A Closer Look at LR(0)

Our LL(1) and LR(0) parsing algorithms differ in the kind of grammars the algorithms can use for parsing input streams.  
We already saw that the grammar,
```
exp : exp + exp
    | x
    | y
```
is not LL(1) because we cannot construct the lookahead sets for this grammar.  But we just proved that it is
LR(0) because we were able to use it in our bottom-up parsing algorithm.
You might be asking at this point: are there LL(1) grammars that are not LR(0)? The answer is yes and surprisingly
the grammar for our Exp0 language,
```
prog : stmt prog
     | ""
          
stmt : p exp ;
     | s var exp ;
     
exp : + exp exp
    | - exp exp
    | ( exp )
    | var
    | num
	
var : x | y | z

num : 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |9
```
is LL(1) as we have shown in the previous sections but not LR(0).
This grammar is not LR(0) due mainly to rule,
```
prog : ""
```
which states that program can be empty.  In bottom-up parsing without lookahead it would be very 
difficult to apply this rule in reverse.  We have two options: (a) either it can be applied anywhere because the empty string can be 
rewritten into `prog` at any time during the derivation or (b) it can never be applied because the empty string
never occurs on the stack explicitely.
It doesn't matter which option we adopt the outcome would be an illegal derivation sequence
and therefore grammars with rules that have empty right sides are not LR(0).
However, it can be shown that grammars with rules that have empty right sides are LR(1), that is, 
using a lookahead allows us to determine whether it is legal to reduce an empty string to the appropriate
non-terminal.

> Grammars that have rules with empty right sides are not LR(0).

<!--
\index{reduce-reduce conflict}
-->

Grammars that have rules with the same right sides are also not considered LR(0).
Consider the following grammar snippet,
```
prog : stmt stmt_list
stmt_list : stmt stmt_list
```
and also consider the following stack configuration,
```
\eos stmt stmt_list
```
At this point our bottom-up parsing algorithm is stuck because it has no way of knowing which rule to pick.
The right sides of both rules match the top of the stack and the algorithm has no way of knowing which rule to 
pick to reduce the stack.
In the terminology of LR(0) parsers this is called a *reduce-reduce* conflict.
Reduce-reduce conflicts show up in grammars with rules that have exactly the same right sides.
We say that,
 
> Grammars with reduce-reduce conflicts are not LR(0).

---
<center>
<img src="figures/chap02/6/figure/Slide1.jpg" alt="">
Fig 6. A trace of the bottom-up parsing algorithm for the input stream `if e then p else q \eof`.
</center>

---


<!--
\index{shift-reduce conflict}
-->

Just as in the case of top-down parsing, grammars with rules whose right sides share common prefixes are not considered LR(0) grammars. 
Consider this grammar snippet,
```
stmt : if exp then stmt
     | if exp then stmt else stmt
     | p
     | q
     
exp : e
```
Figure 6 is a trace of the bottom-up parsing algorithm using this grammar snippet together with the input stream,
```
if e then p else q \eof
```
In the trace the steps 1 through 6 are straightforward with the exception that here we grouped symbols together to make up keywords
and we move the lookahead pointer from input symbol to symbol.
The problem arises in step 7: we can certainly apply the first grammar rule to reduce the top of the stack but then we are stuck with 
a fragment of the if-then-else statement in the input stream for which there are no rules to parse it any further.
The other alternative is to shift the `else` symbol onto the stack and continue parsing.
The is called a *shift-reduce* conflict and we say that,

> A grammar that exhibits shift-reduce conflicts is not LR(0).

We could change the behavior of our algorithm in such a way that if it detects a shift-reduce conflict it always makes the decision to shift
thus avoiding leaving snippets of syntax unused and unparsable in the input stream.
This is in fact what most industrial strength bottom-up parsers do, they issue a warning about a shift-reduce conflict and resolve the conflict
by always shifting.
On the other hand, there is an easy way to rewrite this grammar so that it becomes LR(0) by factoring it.

Similar to LL(1) languages we find that there are multiple grammars that define a language, some of the grammars are LR(0) some are not.  
We can assert the following,

> We call a language LR(0) if it has at least one LR(0) grammar.

<!--
\index{rule postfix}
-->
We should mention that technically LR(0) parsers can run into another kind of reduce-reduce conflict when
the right side of a rule is a postfix of the right side of another rule.  
Consider the grammar,
```
list : list item
     | item
     
item : e
```
Here the right side of the second rule is a postfix to the right side of the first rule.
When parsing the input stream ` e e \eof` with this grammar a pure LR(0) parser has to make a choice when parsing the second
`e` whether to apply the first rule or the second rule in order to reduce the stack to a non-terminal.
If the parser were to choose the second rule the parse will terminate in an error.
In an implementation of LR(0) parsers we could eliminate this choice by forcing the parser to always apply the longest right side of a rule that
matches the stack.  Therefore, in this case the LR(0) parser will always pick the first rule in order to reduce the stack.

## Building Parsers by Hand

<!--
\index{rule application}
-->
If we wanted to build a parser for a language it is clear that we could simply implement either one of the parsers
we just discussed.
However, both of these algorithms spend a significant amount of time searching the grammars for an appropriate rule to apply.
This kind of search is a source of inefficiency.

<!--
\index{recursive descent parser}
-->
It turns out that there is a clever way of turning grammars into parsers where no searching is done whatsoever. 
In this approach to constructing a parser we turn non-terminals on the left side of rules into *function definitions* and non-terminals appearing
within the right side of rules into *functional calls*.  The resulting parser then has a function for each non-terminal in the grammar
and each function knows how to parse the structures associated with its non-terminal.  This kind of parser is called a *recursive descent*
parser because it uses function recursion as a way to make progress during parsing.
Most notably, we require the grammar used to construct a recursive descent parser 
to be an LL(1) grammar with lookahead sets.

Constructing a recursive descent parser is best illustrated with an example.
Consider the following LL(1) grammar with lookahead sets,
```
exp : {+} + exp exp
    | {x,y} var
    
var : {x} x 
    | {y} y
```
This grammar defines two non-terminals, `exp` and `var`, and therefore we expect that the recursive descent parser also has two functions.
The following is the function that parses `exp`,

In [23]:
from grammar_stuff import InputStream

In [24]:
def exp():
    sym = I.pointer()
    if sym == '+':
        I.next()
        exp()
        exp()
    elif sym == 'x':
        var()
    elif sym == 'y':
        var()
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(sym))

Looking at this code carefully you will find that the function  parses expressions according to the 
first two grammar rules above: the grammar rules that define the non-terminal `exp`.
The function first retrieves the
symbol at the lookahead pointer in the input stream.
Then, if this symbol matches the lookahead set of the first rule, *i.e.*, if this lookahead symbol matches the `+` symbol then we execute the first rule.
This means first moving the lookahead pointer to the next character skipping the `+` symbol and then calling `exp()`
recursively (twice -- according to the right side of the first rule).
On the other hand, if the symbol from the lookahead pointer matches either `x` or `y` then we continue parsing the input with the second rule
of the grammar by calling the function `var()`.
If the symbol from the lookahead pointer matches none of the symbols in the lookahead sets then the input stream does not conform
to the structure of an expression and we flag a syntax error.

We can construct a similar function for the non-terminal `var`.

In [25]:
def var():
    sym = I.pointer()
    if sym == 'x':
        I.next()
    elif sym == 'y':
        I.next()
    else:
        raise SyntaxError('unexpected symbol {} while parsing'.format(sym))

As in the previous function this function uses the lookahead sets to select which rule to execute.
In this case the right sides of the rules are very simple, they simply match the variable names in the input stream.
However, if the variable names in the input stream are not correct then the function flags an error and aborts.

Consider parsing the input stream,

In [26]:
I = InputStream(['+','x','y'])

Now we call the parsing function,

In [27]:
exp()

Since it is a syntactically correct expression in the input stream nothing much exciting is happening.

Here is a closer look at the parser.
To start the parser we call the function associated with start symbol of our grammar, `exp`.
This function looks at the symbol under the lookahead pointer, finds a `+` symbol, and uses the first if clause.  Here we move the lookahead pointer to the next symbol in the input stream and then call ourselves
recursively twice.  The first time to match the `x` symbol and the second time to match the `y` symbol.
Notice that there is no searching involved during parsing, everything is completely determined by the lookahead pointer
and the lookahead sets in the case statements.
One way of looking at recursive descent parsers is that the function invocation for a non-terminal activates the right sides of 
the associated rules all at the same time.
The precise right side to execute is then determined by comparing the lookahead pointer and the lookahead set for each right side. 

Let's change the input stream to something that is not syntactically correct,

In [28]:
I = InputStream(['-','x','y'])

In [29]:
exp()

SyntaxError: unexpected symbol - while parsing (<string>)

Sure enough, the parser rejects the expression in the input stream as incorrect.

### Recursive Descent is LL(1)

We know that this parser uses an LL(1) grammar but is it itself an LL(1) parser?  From the discussion above we already know that
a recursive descent parser reads the input from left to right; this gives us our first L.  We also know that it uses a single symbol lookahead
pointer; this gives us our (1).  Finally, the right sides of the grammar rules are encoded in the recursive descent parsers just the way they
are written in the grammar, *i.e.* in the implementation of the rule,
```
exp : {+} + exp exp
```
we first match the `+` symbol and then call the two functions for `exp`,
```
...
if sym == '+':
    I.next()
    exp()
    exp()
...
```
Since code in languages Python (or any imperative language for that matter) is executed top to bottom and
left to right, we know that the function for the first `exp` in the grammar rule is always executed before the second one.
That means the recursive descent parser constructs a left-most derivation.  This gives us our second L.

> Therefore, a recursive descent parser is a LL(1) parser.

<!-- TODO: fix the movie for this
{\bookurl/b/2/q4/figure.mov}
-->


# Parser Generators

<!--
\index{language specification}
\index{language implementer}
-->

Very few language implementers would choose to implement a parser by hand. One reason being that common languages like Java or C typically involve hundreds of 
grammar rules and implementing those in a recursive descent parser by hand 
is a nontrivial undertaking.
Another reason and perhaps the more important reason is that languages evolve during
their lifecycle.  As a language matures new features are added and old features are 
modified or depreciated.
Changing a hand-coded parser to keep up with an evolving language requires major, time consuming reengineering of 
the parser every time the specification of the language changes.

---
<center>
<img src="figures/chap02/7/figure/Slide1.jpg" alt="">
Fig 7. A parser generator viewed as a translator.
</center>

---

<!--
\index{parser generator}
\index{grammar specification}
-->

Most language implementers would choose to use a parser generator in order to construct parsers.
A parser generator reads a grammar specification and generates source code for a parser that can parse
the valid sentences in the language of the specified grammar.
In this way we can view a parser generator as a translator for a domain specific language (see Figure 7).
Here the domain specific language is the grammar specification language and the target language is whatever language
the parser code is generated in.  In our case this will be Python.

The parser generator will go through all the typical stages of a translator.  It first performs a syntax analysis (parsing!) of the 
grammar specification file and constructs an IR.  Then it performs the semantic analysis of the grammar file.
Here it checks whether all non-terminals have been defined by at least one rule, for instance.  Finally, it uses the IR to generate the
parser code.
The advantage of using a parser generator is that as a language evolves all we need to do is to modify the grammar specification,
the parser code is then generated from the modified grammar specification and does not need any additional engineering.

<!--
\index{Ply}
-->

The particular parser generator we will be using throughout the rest of the  book is called 
<a href="http://www.dabeaz.com/ply/">Ply</a> (Python Lex-Yacc).
Ply generates LR(1) parsers from grammar specifications.
The interesting part about Ply is that it is tightly integrated with the Python environment and
generating parsers from the grammar specifications does not require any additional steps from side
of the developer.

Here is the Ply grammar specification for our Exp0 language in the file `exp0_gram.py`,

In [ ]:
# %load code/chap02/exp0_gram.py
from ply import yacc
from exp0_stuff import tokens, lexer

def p_grammar(_):
    """
    prog : stmt_list
    
    stmt_list : stmt stmt_list
              | empty
              
    stmt : 'p' exp ';'
         | 's' var exp ';'
         
    exp : '+' exp exp
        | '-' exp exp
        | '(' exp ')'
        | var
        | num
        
    var : 'x' 
        | 'y' 
        | 'z'
        
    num : '0' 
        | '1' 
        | '2' 
        | '3' 
        | '4' 
        | '5' 
        | '6' 
        | '7' 
        | '8' 
        | '9'
    """
    pass

def p_empty(p):
    'empty :'
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

parser = yacc.yacc()


Besides some housekeeping functions, the grammar specification should look very familiar at this point.
It is essentially the same specification we have been working with.  There is one difference: terminal
symbols are now in quotes.

Please note that grammar specifications have to reside in files. Therefore, we have the magic command
`%load` to load and the display the file.
In order to parse an input stream in our Jupyter notebook we import the parser from the `exp0_gram.py` file
and then run it on some input,

In [31]:
from exp0_gram import parser

Generating LALR tables


Let's parse the program `p + 1 2 ;`,

In [32]:
parser.parse(input="p + 1 2 ;")

As one would expect this was successful since the program is syntactically correct.

<!--
\index{YACC}
-->

Another parser generator that is widely used is YACC (Yet Another Compiler Compiler).
It was developed as part of the original Unix system and became the defacto standard for
LR(1) parser generators.  Notice that Ply gives homage to this lineage in that its parser constructor
is called `yacc()`.
An open source version of YACC is available as Bison at 
<a href="http://www.gnu.org/software/bison">www.gnu.org/software/bison</a>.

## An Example: Our First Language Processor

<!--
\index{language processor}
\index{rvalue}
\index{lvalue}
\index{variable reference}
-->

Let us put this all together and build our first language processor.  Our processor will read an Exp0 program, count the number
of times the program references the value of a variable, and then prints the number of value references to the terminal.
In Exp0 the only place where we can access the value of a variable is within an expression -- see the Exp0
grammar in the previous section.
Notice that the occurrence of a variable within an expression is very different from a variable occurrence within a `store` statement (at least when in occurs as the first argument to the `store` statement).
When we have a variable reference within an expression then we are interested in retrieving the value that is stored in the variable.
On the other hand, if we have a variable reference as the first argument to the `store` statement then we are interested in
changing the value that is stored in the variable.
In the traditional terminology of language processors we often refer to variable references within expressions as *rvalues* and  
variable references that allow us to change the value of the variable as *lvalues*.
This terminology is based on the fact that variable references to the right of an assignment operator access the value of a variable, the rvalue, and
variable reference to the left of an assignment operator modify the storage location of the variable, the lvalue.

Given these two distinct ways of referencing variables and given that in this case we are only interested in rvalues we have to build a language processor that understands the difference between lvalues and rvalues.  
That means our language processor has to parse the input and only count the variable occurrences within expressions and ignore the lvalues.
A program that does simple pattern matching on the variable names will not work here.

Here is the grammar specification,

In [ ]:
# %load code/chap02/exp0_count.py
from ply import yacc
from exp0_stuff import tokens, lexer

count = 0

def p_prog(_):
    '''
    prog : stmt_list
    '''
    print("count = {}".format(count))
    
def p_stmt_list(_):
    '''
    stmt_list : stmt stmt_list
              | empty
    '''
    pass

def p_stmt(_):
    '''
    stmt : 'p' exp ';'
         | 's' var exp ';'
    '''
    pass
         
def p_exp(_):
    '''
    exp : '+' exp exp
        | '-' exp exp
        | '(' exp ')'
        | num
    '''
    pass

def p_exp_var(_):
    '''
    exp : var
    '''
    global count
    count += 1

def p_var(_):
    '''
    var : 'x' 
        | 'y' 
        | 'z'
    '''
    pass
        
def p_num(_):
    '''
    num : '0' 
        | '1' 
        | '2' 
        | '3' 
        | '4' 
        | '5' 
        | '6' 
        | '7' 
        | '8' 
        | '9'
    '''
    pass

def p_empty(p):
    'empty :'
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

def init_count():
    global count
    count = 0

parser = yacc.yacc()


Here we broke the grammar into smaller chunks so that we can attach code (called *embedded actions*) to the individual rules.  In particular, the first rule that defines the non-terminal `prog`.  A soon as this is parsed we print
out the count which is a count of the number of times we saw a variable in an expression.
That brings us to the other embedded action for the rule `exp : var`.  Everytime we use this rule to parse a 
variable in an expression we use the embedded action to increment our count.  There is no problem with confusing
lvalues and rvalues for variables.

Let's run this simple language processor on some sample input,

In [34]:
from exp0_count import parser, init_count

Generating LALR tables


In [35]:
init_count()
parser.parse(input="s x + y 1 ;")

count = 1


Our processor did the right thing.  Even though there are two variables in the source program our processor only
counted the occurrence in the expression.

Let's try something a bit more ambitious.

In [36]:
init_count()
parser.parse(input=
'''
s x 1;
p x;
s y 2;
p y;
p (+ x y);
''')

count = 4


Our processor correctly identified four rvalue variable references.
Documentation for Ply can be found <a href="http://www.dabeaz.com/ply/ply.html">here</a> and a slightly more
elaborate example in the form of a calculator language can be found <a href="http://www.dabeaz.com/ply/example.html">here</a>.

# Lexical Analysis


<!--
\index{lexical structure}
\index{lexical rule}
\index{token}
-->

The language generated by our Exp0 grammar only allows single character words, that is, keywords,  variable names, and numbers are all restricted to a single character.
However, real programming languages allow longer words as part of their syntax.
This introduces the notion of lexical structure of a programming language.
Furthermore, the lexical structure of a programming language is 
specified with rules similar to grammar rules.  These rules are called *regular expressions*.

The lexical specification of a programming language defines how individual symbols are combined to form words.
These words are then grouped into tokens which are the entities that parser generators work with when converting a grammar
specification into parsers.
The tokens defined in lexical rules act similarly to non-terminals in grammar rules in that they summarize
the lexical structure of a group of related words.  Parser generators use the lexical rules to generate *lexers* that group symbols into tokens.  Figure 8 shows how the lexical structure of a programming language is related to the phrase structure of that language.

---
<center>
<img src="figures/chap02/8/figure/Slide1.jpg" alt="">
Fig 8. The specification of the syntax of a programming language.
</center>

---


Given the lexical structure of programming languages, parsing now proceeds conceptually as a two stage process.  First the input stream is tokenized by the lexer and the tokens are sent 
to the parser as a token stream in order to recognize the phrase structure of the program.

In order to get a feel of what a lexical specification looks like we will work through a simple example from the
<a href="http://www.dabeaz.com/ply/ply.html#ply_nn4">Ply documentation</a>.  The following is a Ply specification
of a lexer for a simple arithmetic language.

In [ ]:
# %load code/chap02/calc_lex.py
import ply.lex as lex

# List of token names.   This is always required
tokens = (
   'NUMBER',
   'PLUS',
   'MINUS',
   'TIMES',
   'DIVIDE',
   'LPAREN',
   'RPAREN',
)

# Regular expression rules for simple tokens
t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_LPAREN  = r'\('
t_RPAREN  = r'\)'

# A regular expression rule with some action code
def t_NUMBER(t):
    r'[0-9]+'
    t.value = int(t.value)    
    return t

# Define a rule so we can track line numbers
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# A string containing ignored characters (spaces and tabs)
t_ignore  = ' \t'

# Error handling rule
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)

# Build the lexer
lexer = lex.lex()



We begin the specification by listing all the token names for our language.  Here we see tokens for numbers as well as operators of our language.  Next we define the lexical structure of each of the tokens.
These rules are very similar to grammar rules in the sense that the token names act as non-terminals and the right
side of the rules define the structure attached to these names.  In this case the structure for these tokens consists
of single characters defining the operators. We use Python raw strings here so we can use single escape backslashes.

Next we define the lexical structure of tokens that need additional embedded actions.
Take for example the token `t_NUMBER`.  The structure of this token is defined by the regular expression `[0-9]+`.
This regular expression makes use of the reserved class `[0-9]` which represents all digits.  The plus sign
specifies that the token `t_NUMBER` consists of 'one or more' digits.
We then convert the string representing the value into an actual integer value and attach that value 
to the token structure.

Finally, we define that lexical structure of three built-in tokens: `t_newline`, `t_ignore`, and `t_error`.
Each lexer specification has to provide these. On the last line we construct the lexer object.

Let's test drive this lexer.

In [38]:
from calc_lex import lexer

In [39]:
input_stream = '22 * 3 + 45'

In [40]:
lexer.input(input_stream)

In [41]:
for tok in lexer:
    print(tok)

LexToken(NUMBER,22,1,0)
LexToken(TIMES,'*',1,3)
LexToken(NUMBER,3,1,5)
LexToken(PLUS,'+',1,7)
LexToken(NUMBER,45,1,9)


Here we see that the token stream consists of five tokens.
As expected the token `TIMES` has the value '`*`' and the token `PLUS` has the value '`+`'.
More interesting are the `NUMBER` tokens.  We see that the first `NUMBER` token has the value 22, the second
the value 3, and the third the value 45.  Which is consistent with the input stream.

This illustrates that the same token class can take on different values depending on the input stream.

## An Example: The Exp1 Language

Let's extend Exp0 to include multi-character keywords and variable names as well as numbers that contain more than a single digit.
The following is the grammar specification of Exp1.

In [ ]:
# %load code/chap02/exp1_gram.py
from ply import yacc
from exp1_lex import tokens, lexer

def p_grammar(_):
    """
    prog : stmt_list
    
    stmt_list : stmt stmt_list
              | empty
              
    stmt : PRINT exp ';'
         | STORE var exp ';'
         
    exp : '+' exp exp
        | '-' exp exp
        | '(' exp ')'
        | var
        | num
        
    var : NAME
        
    num : NUMBER
    """
    pass

def p_empty(p):
    'empty :'
    pass

def p_error(t):
    print("Syntax error at '%s'" % t.value)

parser = yacc.yacc()


Structurally the grammar looks very similar to the Exp0 grammar but we have inserted tokens for the `store` and `print` keywords as well as for variable names and numbers.

Let's take a look at the lexer for this language.

In [ ]:
# %load code/chap02/exp1_lex.py
# Lexer for Exp1

from ply import lex

reserved = {
    'store' : 'STORE',
    'print' : 'PRINT'
}

literals = [';','+','-','(',')']

tokens = ['NAME','NUMBER','NEWLINE'] + list(reserved.values())

t_ignore = ' \t'

def t_NAME(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = reserved.get(t.value,'NAME')    # Check for reserved words
    return t

def t_NUMBER(t):
    r'[0-9]+'
    t.value = int(t.value)
    return t

def t_NEWLINE(t):
    r'\n'
    t.lexer.lineno += 1

def t_error(t):
    print("Illegal character %s" % t.value[0])
    t.lexer.skip(1)

# build the lexer
lexer = lex.lex()


There should not be any surprises here.  The most complicated regular expression is the lexical definition
of a name. The regular expression,
```
[a-zA-Z_][a-zA-Z_0-9]*
```
states that names start with either a lower case character, upper case character, or underscore.  This is followed by zero or more (`*`) lower or upper case characters or numbers or an underscore.

Let's test this language specification.

In [44]:
from exp1_gram import parser

In [45]:
input_stream = "store x1 10 ; print + x1 1 ;"

In [46]:
parser.parse(input=input_stream)

It worked!  Notice that Exp1 is starting to look like a real programming language.

<!--
\index{structural token}
-->

A closer look at our lexical specification reveals that we have two kinds of tokens: One kind of token consists only of a single 
word
(*e.g.*, consider the token `PRINT`, the only word belonging to this token is the word `print`).
We call tokens that only consist of a single word *structural tokens* and they typically represent the keywords and operators of the programming language being specified.
The other kind of
token has a possibly infinite number of words (*e.g.*, consider the token `NUMBER`).
These tokens usually represent values or other entities that are not rigidly define in the language.

## Regular Expressions 

<!--
\index{regular expression}
-->

In the previous section we informally introduced regular expressions as a way to specify the structure of words belonging
to a token.  It is time to formalize this.  Here is definition of most common regular expressions,

1. Each letter `A` through `Z` and `a` through `z` is a regular expression.

2. Each number `0` through `9` is a regular expression.

3. Each printable character `(`, `)`, `-`, `+`, *etc.* is a regular expression.

4. If *A* and *B* are regular expressions then *AB* is also a regular expression and represents the concatenation
 of the two regular expressions.
 
5. If *A* is a regular expression then `(`*A*`)` is also a regular expression.  Parentheses allow us to group regular expressions.  If we wanted to have parentheses be part of the regular expression itself we would have to 'escape' them.  That is, `(`*A*`)` is different from `\(`*A*`\)`.

6. If *A* and *B* are regular expressions then *A*`|`*B* is also a regular expression and represents the choice between
 regular expressions: *A* or *B*.

7. If *A* is a regular expression then *A*`?` is also a regular expression and specifies the regular expression *A* as optional.

8. If *A* is a regular expression then *A*`*` is also a regular expression and specifies that the regular expression *A* can appear zero or more times.

9. If *A* is a regular expression then *A*`+` is also a regular expression and specifies that the regular expression *A* can appear one or more times. 

In addition to these definitions of regular expressions many tools provide extensions that make
the definition of tokens easier.  The Python regular expression syntax also defines the following
regular expressions:

1. The regular expression `[A-Z]` represents a single character between `A` and `Z`.  Similarly for `[a-z]` and `[0-9]`.

2. The dot operator is a regular expression that represents any single printable character.

3. The special characters `\n`, `\t`, and `\r` are also regular expressions
representing the newline character, the tab character, and the carriage return character, respectively.

4. The `^` operator computes the complement of a set.  For example, if we have the regular expression `[abc]` matching either `a`, or `b`, or `c`, then the complement `[^abc]` will match any character other than `a`, `b`, or `c`.  This is useful in
conjunction with character classes.  For example, the regular expression 
`[A-Z][^A-Z]`
specifies a word structure that starts with a capital letter followed by a single character that is not a capital letter.

More details on regular expressions can be found in 
<a href="https://docs.python.org/3/library/re.html">Python's regular expression documentation</a>.

Just to exercise this machinery a bit here is a regular expression for integer values,
```
0 | -?[1-9][0-9]*
```
First notice that this expression consists of two regular expressions, namely `0` and  `-?[1-9][0-9]*`,
joined together by the `|` operator denoting a choice.  
The first regular expression is just the character `0` denoting the
fact that one choice is just the digit `0`.  The second regular expression is the concatenation of three different 
regular expressions: `-?`, `[1-9]`, and `[0-9]*`.  The first one denotes an optional minus sign, the second
denotes a single digit between 1 and 9, and the last one denotes zero or more digits between 0 and 9.
The concatenation of the three regular expressions represents possibly negative numbers that start with a digit between 1 and 9 followed 
by zero or more digits between 0 and 9.

# Summary

We started this chapter with a discussion of grammars and defining languages using grammars.  Derivations within grammars allow
us to show that a particular sentence is a valid sentence and belongs to the language of the grammar.  
Once we have a derivation it is straightforward to construct a parse tree and we can view parse trees as a visual representation of
derivations.

The construction of derivations can be mechanized using a parsing algorithm.  We classify parsing algorithm according to how they
construct the associated derivation or parse tree: Top-down -- starting at the grammar start symbol.  Bottom-up -- starting at the leaves
of the parse tree.  Top-down parsing algorithms are also called LL(k) algorithms in that they read the input from the (L)eft, construct
the (L)eft-most derivation, and use (k) lookahead symbols.  Bottom-up parsing algorithms are also called LR(k) algorithms because
they read the input from the (L)eft, build the (R)ight-most derivation, and use (k) lookahead symbols.  Here we studied the LL(1) and LR(0)
parsing algorithms in more detail.
We also introduced the parser generator Ply we will be using for the remainder of this book.  Ply generates LR(1)
parsers.

Practical parsing of programming languages is done as a two step process: First we perform the lexical analysis, that is, we analyze the structure
of the language words.  Then we perform the syntactic or phrase structure analysis of the program.
The phrase or syntactical structure of a programming language is given using grammar rules while the lexical structure is specified using 
regular expressions.

# Bibliographic Notes

A number of books introduce formal language theory, grammars, and derivations within grammars in addition to finite state machines.
Perhaps the most accessible books are
by Webber (Webber, 2008) and Sipser (Sipser, 2012).
Most books on compiler construction discuss parsing theory in detail and introduce efficient algorithms for lexical and syntax analysis, *e.g.*, (Aho, 1986) and (Cooper & Torcson, 2011).
Donald Knuth's seminal paper (Knuth, 1965) introduced LR parsing in 1965 and made parsing practical and efficient.
Grammars with respect to operator associativity and precedence are discussed in (Webber, 2010).

Webber, A. B. (2008). *Formal language: A practical introduction*. Franklin, Beedle & Associates Inc.

Sipser, M. (2012). *Introduction to the Theory of Computation*. Cengage Learning.

Aho, A. V., Sethi, R., & Ullman, J. D. (1986). *Compilers, Principles, Techniques*. Boston: Addison-Wesley.

Cooper, K., & Torczon, L. (2011). *Engineering a compiler*. Elsevier.

Knuth, D. E. (1965). *On the translation of languages from left to right*. Information and control, 8(6), 607-639.

Webber, A. B. (2010). *Modern programming languages: A practical introduction.* Franklin, Beedle & Associates Inc.


# Exercises

1. Define a small LL(1) language using a grammar.  Write the grammar in tuple notation as we did
in the section on lookahead sets.  Then use the lookahead set algorithm to compute the grammar extended
with the lookahead sets.

2. Define a small left-recursive language.  Write its grammar in tuple notation as we did in the section on lookahead sets.  Convince yourself that the lookahead set algorithm does not work
on this grammar. Is it LR(0)?

3. Use the Exp0 grammar in tuple notation and the top-down parsing algorithm and show that you can parse
some non-trivial Exp0 programs.

4. LL(0) parsers are parsers that read input from left to right, construct the left-most derivation but use no lookahead symbols.
What does that mean for the kind of grammars such a parser could use for parsing?  

5. Rewrite the lookahead set algorithm in such a way that it detects left-recursion, issues an error message, and terminates. The extension should be general enough so that it also detects left-recursion in mutually recursive rules.

6. We have shown that if-then-else statements create shift-reduce conflicts in LR(0) parsers.  Show that right-recursive rules such as
  ```
  list : item
        | item list
       
  item : e
  ```
also create a shift-reduce conflict in our LR(0) parser.
Compute a trace similar to Figure 6 for the input stream `e e e \eof`.

7. Design a lexical rule that defines a token for the binary representation of positive integer numbers.

8. Design a lexical rule defines a token for the floating point numbers (don't include the scientific notation).

9. Write the following grammar as a Ply specification (including both lexical and syntax specifications), and parse both correct and incorrect input streams.
  ```
  list : [ element_list ]
  
  element_list : element_list , element
                | element
  	
  element : a | b | c
  ```

10. Write a grammar for the arithmetic expressions in your favorite programming language.

11. (project) Extend the top-down parsing algorithm so that it constructs and returns a parse tree.

12. (project) Extend the bottom-up parsing algorithm so that it constructs and returns a parse tree.
 
13. (project)  Extend the bottom-up parsing algorithms so that it flags the all rules that either create an LR(0) reduce-reduce conflict or a shift-reduce conflict.
 
14. (project) Reduce-reduce conflicts in bottom-up parsers can be resolved by guessing and backtracking: guess one alternative and try to parse the sentence; if this is not possible backtrack to the decision point and try another alternative. Extend the bottom-up parsing algorithm with backtracking on reduce-reduce conflicts and implement a parser using your extended algorithm that reads a grammar  and an input stream and returns `True` if the program in the input stream is a valid sentence in the language of the grammar otherwise `False`.

15. (project) Build a recursive descent parser for the Exp0 programming language.  Show that it works by parsing some sentences that are valid in Exp0.  Show that it rejects invalid sentences with syntax errors.

16. (project) Modify your recursive descent parser from Exercise 15 to construct parse trees for input programs.
 